In [1]:
import sys

In [3]:
sys.path

['C:\\Users\\User\\Desktop\\מדמח\\מודלימז\\פרויקט חדש',
 'C:\\Users\\User\\Downloads\\pytorch-normalizing-flows-master\\pytorch-normalizing-flows-master\\nflib',
 'C:\\Users\\User\\try',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\plotbrowser-0.1-py3.9.egg',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\Pythonwin']

In [2]:
sys.path.append(r"c:\users\user\anaconda3\envs\pytorch_env\lib\site-packages")


In [3]:
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
import time
from util import findStatistics
from sklearn.preprocessing import LabelEncoder

In [8]:
df=pd.read_csv(r"C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\archive (s3)\milknew.csv")




# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['Grade']= label_encoder.fit_transform(df['Grade'])
  
df['Grade'].unique()
X = df[['pH', 'Temprature', 'Taste', 'Odor', 'Fat ', 'Turbidity', 'Colour']]
y = df[['Grade']]


from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
 
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
 
# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
 
# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [11]:
parameters = {'max_depth':list(range(50)), 'min_samples_leaf':list(range(2,50,1)),'criterion':["gini", "entropy"]}

et=time.time()
model= DecisionTreeClassifier()
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)
ft=time.time()

print("elapsed time {}".format(ft-et))
print("score: {}".format(clf.score(X_test,y_test)))

elapsed time 50.56204390525818
score: 0.9811320754716981


In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(15,32)
        self.fc2=nn.Linear(32,64)
        self.fc6=nn.Linear(64,128)
        self.fc7=nn.Linear(128,256)
        self.fc8=nn.Linear(256,128)
        self.fc3=nn.Linear(128,32)
        self.fc4=nn.Linear(32,8)
        self.fc5=nn.Linear(8,1)


    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc3(x))
        
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x
    
     
    
    
net = Net()
net.load_state_dict(torch.load(r'C:\Users\User\Desktop\מדמח\מודלימז\פרויקט חדש\batches117'))
#best_max_depth,best_min_samples_leaf,best_criterion=params[np.argmax(scores)]
#best_criterion='entropy' if best_criterion==1 else 'gini'

<All keys matched successfully>

In [16]:
y_train=np.array(y_train)

In [18]:
scores=[]

et=time.time()
statistics=np.array(findStatistics(X_train,y_train))
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            hyperTensor=np.array([max_depth,min_samples_leaf,criterion])
            combtensor=torch.tensor(np.concatenate((statistics,hyperTensor))).float()
            
            score_predicted=net(combtensor).detach().numpy()
            scores.append(score_predicted)
params=[]
for max_depth in list(range(50)):
    for min_samples_leaf in list(range(2,50,1)):
        for criterion in [1, 2]:
            params.append((max_depth,min_samples_leaf,criterion)) 
            
scores=np.array(scores).flatten()
params=np.array(params)

tenLargetIdx = (-scores).argsort()[:10]
bests=params[tenLargetIdx]   


res=[]
for max_depth,min_samples_leaf,criterion in bests:
    criterion='entropy' if criterion==1 else 'gini'
    model= DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
    model.fit(X_train,y_train)
    res.append(model.score(X_test,y_test))
    

best=np.argmax(np.array(res))
best_max_depth,best_min_samples_leaf,best_criterion=bests[best]
best_criterion='entropy' if best_criterion==1 else 'gini'
model2= DecisionTreeClassifier(max_depth=best_max_depth,min_samples_leaf=best_min_samples_leaf,criterion=best_criterion)

model2.fit(X_train,y_train)
ft=time.time()
print("elapsed time {}".format(ft-et))

print('score: {}'.format(model2.score(X_test,y_test)))
            


elapsed time 2.148998498916626
score: 0.9874213836477987


In [17]:
best_max_depth

8